__1. Warm-up to model__

In [1]:
import numpy as np
import pandas as pd


/home/van.kh/.local/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/van.kh/.local/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [ ]:

!pip install torch


In [3]:
import torch
import torch.nn as nn

import torch.optim as optim
from torch.optim.lr_scheduler import StepLR


In [ ]:
!pip install datasets==2.18.0
!pip install sentencepiece



In [ ]:
!pip install accelerate -U

In [4]:
from datasets import load_dataset
import pandas as pd
dataset = load_dataset("cnn_dailymail", "3.0.0")
dataset



/shared/centos7/anaconda3/2022.05/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [4]:
def show_samples(dataset, num_samples=3, seed=42):
    sample = dataset["train"].shuffle(seed=seed).select(range(num_samples))
    for example in sample:
        print(f"\n'>> Article: {example['article']}'")
        print(f"'>> Summary: {example['highlights']}'")


show_samples(dataset)


'>> Article: By . Anthony Bond . PUBLISHED: . 07:03 EST, 2 March 2013 . | . UPDATED: . 08:07 EST, 2 March 2013 . Three members of the same family who died in a static caravan from carbon monoxide poisoning would have been unconscious 'within minutes', investigators said today. The bodies of married couple John and Audrey Cook were discovered alongside their daughter, Maureen, at the mobile home they shared on Tremarle Home Park in Camborne, west Cornwall. The inquests have now opened into the deaths last Saturday, with investigators saying the three died along with the family's pet dog, of carbon monoxide poisoning from a cooker. Tragic: The inquests have opened into the deaths of three members of the same family who were found in their static caravan last weekend. John and Audrey Cook are pictured . Awful: The family died following carbon monoxide poisoning at this caravan at the Tremarle Home Park in Camborne, Cornwall . It is also believed there was no working carbon monoxide detec

In [5]:
#Filter articles and highlights too long
def filter_examples(example):
    article_words = len(example['article'].split())
    highlight_words = len(example['highlights'].split())
    return article_words <= 1200 and highlight_words <= 500

# Apply the filter function to the dataset
filtered_dataset = dataset['train'].filter(filter_examples)

# Print the number of examples in the filtered dataset
print("Number of examples after filtering:", len(filtered_dataset))


Number of examples after filtering: 261525


In [6]:
#same for validation 
filtered_val = dataset['validation'].filter(filter_examples)
print("Number of examples after filtering:", len(filtered_val))

Number of examples after filtering: 12161


In [ ]:
!pip install transformers

In [7]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

model_checkpoint = "Falconsai/text_summarization"
tokenizer = T5Tokenizer.from_pretrained(model_checkpoint)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
example_input = tokenizer('I am working on my project.')
example_input


{'input_ids': [27, 183, 464, 30, 82, 516, 5, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [9]:
tokenizer.convert_ids_to_tokens(example_input.input_ids)

['▁I', '▁am', '▁working', '▁on', '▁my', '▁project', '.', '</s>']

__2. Preprocessing__

In [8]:
max_input_length = 1200
max_target_length = 500


def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["article"],
        max_length=max_input_length,
        truncation=True,
    )
    labels = tokenizer(
        examples["highlights"], max_length=max_target_length, truncation=True
    )
    model_inputs["labels"] = labels["input_ids"]
    model_inputs["max_new_tokens"] = [max_input_length] * len(model_inputs["input_ids"])

    return model_inputs

In [9]:
tokenized_train = filtered_dataset.map(preprocess_function, batched=True)

In [10]:
tokenized_val = filtered_val.map(preprocess_function, batched=True)

__3. Define the evaluation method__

In [ ]:
!pip install rouge_score
!pip install evaluate


In [11]:
import evaluate
rouge_score = evaluate.load("rouge")


__4. Finetuning__

In [12]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [13]:
from huggingface_hub import notebook_login

notebook_login()

In [18]:
article = filtered_dataset['article'][0]
summary = filtered_dataset['highlights'][0]
tokenized_article = tokenizer(article, truncation=True, padding=True, return_tensors="pt")
output = model.generate(**tokenized_article)
generated_summary = tokenizer.decode(output[0], skip_special_tokens=True)
print('Reference:',summary)
print('Output:',generated_summary)

Reference: Harry Potter star Daniel Radcliffe gets £20M fortune as he turns 18 Monday .
Young actor says he has no plans to fritter his cash away .
Radcliffe's earnings from first five Potter films have been held in trust fund .
Output: : Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($


In [14]:
from transformers import Seq2SeqTrainingArguments

batch_size = 8
num_train_epochs = 2
# Show the training loss with every epoch
logging_steps = len(tokenized_train) // batch_size
model_name = model_checkpoint.split("/")[-1]

args = Seq2SeqTrainingArguments(
    output_dir=f"{model_name}-cnn",
    evaluation_strategy="epoch",
    learning_rate=5.6e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=num_train_epochs,
    predict_with_generate=True,
    logging_steps=logging_steps,
    push_to_hub=True,
)

In [15]:
import nltk
#nltk.download("punkt")
from nltk.tokenize import sent_tokenize

In [16]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Decode generated summaries into text
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # Decode reference summaries into text
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # ROUGE expects a newline after each sentence
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
    # Compute ROUGE scores
    result = rouge_score.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    # Extract the median scores
    rouge_scores = {}
    for key, value in result.items():
        if hasattr(value, 'mid'):
            rouge_scores[key] = value.mid.fmeasure * 100
        else:
            rouge_scores[key] = value * 100  # Handle float values directly
    
    return {k: round(v, 4) for k, v in result.items()}

In [17]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [18]:
tokenized_train = tokenized_train.remove_columns(
    filtered_dataset.column_names
)

In [24]:
features = [tokenized_train[i] for i in range(2)]
data_collator(features)

{'input_ids': tensor([[  301, 24796,  4170,  ...,     0,     0,     0],
        [11953,    31,     7,  ...,     3,     5,     1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1]]), 'labels': tensor([[ 8929, 16023,  2213,  4173,  6324, 12591,    15,  2347,  3996,  1755,
           329, 13462,    38,     3,    88,  5050,   507,  2089,     3,     5,
          5209,  7556,   845,     3,    88,    65,   150,  1390,    12,  9030,
            17,   449,   112,  1723,   550,     3,     5,  6324, 12591,    15,
            31,     7,  8783,    45,   166,   874, 16023,  4852,    43,   118,
          1213,    16,  2019,  3069,     3,     5,     1,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100],
        [17054,   120,     3,  1092,    16, 11171,    16,  8327,    33,   629,
            26,    30,     8,    96,  1161,  7483,  1501,   121, 12330, 

In [19]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/home/van.kh/.local/lib/python3.9/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [20]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,1.806200,1.626197,0.248000,0.119800,0.205300,0.234000
2,1.756300,1.612001,0.248300,0.120300,0.205500,0.234400


/home/van.kh/.local/lib/python3.9/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/home/van.kh/.local/lib/python3.9/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/home/van.kh/.local/lib/python3.9/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/home/van.kh/.local/lib/python3.9/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic defau

TrainOutput(global_step=65382, training_loss=1.7812006839180594, metrics={'train_runtime': 17397.565, 'train_samples_per_second': 30.065, 'train_steps_per_second': 3.758, 'total_flos': 1.6324124343002726e+17, 'train_loss': 1.7812006839180594, 'epoch': 2.0})

In [ ]:
trainer.evaluate()

/home/van.kh/.local/lib/python3.9/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


/home/van.kh/.local/lib/python3.9/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/home/van.kh/.local/lib/python3.9/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/home/van.kh/.local/lib/python3.9/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/home/van.kh/.local/lib/python3.9/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic defau

In [21]:
trainer.push_to_hub(commit_message="Training complete", tags="summarization")

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/vmkhoa2000/text_summarization-cnn/commit/8d45fe76f38bb00c7a1f638195821709f746d4ec', commit_message='Training complete', commit_description='', oid='8d45fe76f38bb00c7a1f638195821709f746d4ec', pr_url=None, pr_revision=None, pr_num=None)